In [15]:
import tweepy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import psycopg2
import datetime as dt
from tabulate import tabulate
import requests
import base64

pd.set_option("display.max_columns", None)


In [2]:
listado_precios = pd.read_csv('../airflow/data/precios.csv', sep=',', encoding='latin-1')

lista_variaciones = {}
for producto in listado_precios:
    if producto == 'fecha':
        continue
    elif producto == 'id':
        continue
    else:
        variacion = (listado_precios[producto].iloc[-1] / listado_precios[producto].iloc[0] - 1) * 100
        lista_variaciones[producto] = variacion
        
precio_anterior = sum(listado_precios.iloc[0, 2:])
precio_actual = sum(listado_precios.iloc[-1, 2:])
print(f"El precio anterior es ${precio_anterior}")
print(f"El precio actual es ${precio_actual}")
suma = sum(lista_variaciones.values()) / len(lista_variaciones)
print('La variación media de precio de los productos de la canasta básica es del', round(suma, 2), '%')

El precio anterior es $43354.810000000005
El precio actual es $52402.0
La variación media de precio de los productos de la canasta básica es del 26.66 %


In [3]:
conn = psycopg2.connect(
    host="canasta-2.crdqtsbdpist.us-east-2.rds.amazonaws.com",
    database="postgres",
    user="postgres",
    password="postgres"
)
# obtengo la fecha de hoy

fecha = dt.datetime.now().strftime("%Y-%m-%d")
primer_dia_mes_actual = dt.datetime.now().replace(day=1)
# traigo de la base de datos la última fecha 

query = f"""
SELECT *
FROM precios_lista_larga
WHERE fecha = '{fecha}'
"""

df = pd.read_sql(query, conn)


# Traigo los precios de la base de datos del prime dia del mes actual

query = f"""
SELECT *
FROM precios_lista_larga
WHERE fecha = '{primer_dia_mes_actual}'
"""

df2 = pd.read_sql(query, conn)

conn.close()

# armo una funcion de esto






C:\Users\ismael\AppData\Local\Temp\ipykernel_22852\2461419181.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\ismael\AppData\Local\Temp\ipykernel_22852\2461419181.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query, conn)


In [4]:
fecha_actual = df
primer_dia_mes = df2
print(fecha_actual.info())
print(primer_dia_mes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     63 non-null     object 
 1   producto  63 non-null     object 
 2   precio    63 non-null     float64
 3   id        63 non-null     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     63 non-null     object 
 1   producto  63 non-null     object 
 2   precio    63 non-null     float64
 3   id        63 non-null     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.1+ KB
None


In [5]:
# Si un producto de fecha_actual tiene precio 0, lo remuevo de primer_dia_mes

for producto in fecha_actual['producto']:
    if fecha_actual[fecha_actual['producto'] == producto]['precio'].iloc[0] == 0:
        primer_dia_mes.drop(primer_dia_mes[primer_dia_mes['producto'] == producto].index, inplace=True)
    
# remuevo los productos con precio 0 de fecha_actual

fecha_actual.drop(fecha_actual[fecha_actual['precio'] == 0].index, inplace=True)






In [6]:
# sumo los precios de fecha_actual y primer_dia_mes

precio_anterior = sum(primer_dia_mes['precio'])
precio_actual = sum(fecha_actual['precio'])

# calculo la variación

variacion = (precio_actual / precio_anterior - 1) * 100

# obtengo el nombre del mes en español y numero del dia




nombre_mes_completo = fecha_actual['fecha'].iloc[0].strftime("%B")
# paso mes a español

meses = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

nombre_mes = meses[nombre_mes_completo]

# obtengo el numero del dia y el nombre del dia

numero_dia_y_nombre = fecha_actual['fecha'].iloc[0].strftime("%d")

numero_dia_y_numero = fecha_actual['fecha'].iloc[0].strftime("%d")


print(f"La variacion de precios de la canasta básica en el mes de {nombre_mes} al dia {numero_dia_y_nombre} es del {round(variacion, 2)}%")

twit = f"La variacion de precios de la canasta básica en el mes de {nombre_mes} al dia {numero_dia_y_nombre} es del {round(variacion, 2)}%"

La variacion de precios de la canasta básica en el mes de Mayo al dia 27 es del 8.59%


Twitear el resultado

In [25]:
# credenciales de la api de twitter



access_token = "1662449848463900672-RJoVpQCKmM3MSjpgNlUqpw0yDejl2x"
secret_token = "5KQlav9mP8s9XNhHIGW2doKc7GTlHgJXANNFj28kOsbyX"
api_key = "pEAHT22z6YGad3iKo2y5S74SK"
api_secret_key = "SQPGdU9bRz8xygsADdtvXEzQl3OctRwain7ftNvjFJ1aqIcQ6s"
bearer_token = ""
client_id = "aVBFc2FDTzFRYVFHNkhmdjkycVU6MTpjaQ"
client_secret = "joDm6JJ7nsKyUbaE28yTdYiGd01DeAFpiZA1978mWi_E_u-neF"

# # twiteo con tweepy

# client = tweepy.Client(bearer_token=bearer_token, consumer_key=api_key, consumer_secret=api_secret_key, access_token=access_token, access_token_secret=secret_token)

# client.create_tweet(text=twit)


# twiteo con requests creo una funcion para esto y uso la api v2

bearer_token = "AAAAAAAAAAAAAAAAAAAAAJ81nwEAAAAA4e%2BlUsyU5qUWKW81yZaVpXYFuzE%3Dm2YumulAw6QiCLY5LIMXyxpIdsQQW2nLTXmqWFwOFwS62VBpxY"

def obtener_tweets():
    url = "https://api.twitter.com/2/tweets"

    headers = {
        "Authorization": f"Bearer {bearer_token}"
    }

    params = {
        "ids": "tweet_id_1,tweet_id_2",  # Reemplaza con los IDs de los tweets que deseas obtener
        "tweet.fields": "created_at,public_metrics"  # Campos de los tweets que deseas incluir en la respuesta
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("No se pudo obtener los tweets")

# Ejemplo de uso
tweets = obtener_tweets()
print(tweets)


Exception: No se pudo twittear